In [44]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 200)
pd.options.display.float_format = '{:,.0f}'.format

In [81]:
columns = ['id','scrape_id', 'neighbourhood', 'zipcode', 'property_type', 'room_type', 'accommodates', 'bedrooms', 'beds', 'price', 'number_of_reviews', 'review_scores_rating', 'reviews_per_month'] 
listings = pd.read_csv('listings.csv')
listings = listings[columns]
listings.index = listings['id']
del listings['id']
listings['price'] = listings['price'].replace(['[\$]', '[\,]'], '', regex=True).apply(pd.to_numeric)
# --> filter zipcode on first 4 numbers
listings['zipcode'] = listings['zipcode'].str[:4].apply(pd.to_numeric, errors='coerce')
listings.dropna(subset=['zipcode'], inplace=True)
# convert dat zipcode to a prupper int
listings['zipcode'] = listings['zipcode'].apply(lambda x: int(x))
listings['zipcode'].iloc[0]

1054

In [82]:
#price & reviews 
listings['zipcode'].astype('category')
price_reviews = pd.pivot_table(listings, index=['zipcode'], values=['price', 'scrape_id','number_of_reviews', 'review_scores_rating', 'reviews_per_month'], aggfunc={'price': np.mean, 'scrape_id': len, 'review_scores_rating': np.mean, 'reviews_per_month': np.mean, 'number_of_reviews': np.sum})
del price_reviews['scrape_id']
# --->>> JSON
price_reviews.sort() # SORT
price_reviews.to_json('price_reviews.json', orient='index')

/Users/sjoerd/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: sort(....) is deprecated, use sort_index(.....)


In [76]:
# PROPERTY TYPES PIVOT

property_types = pd.pivot_table(listings, index='zipcode', columns='property_type', values='scrape_id', aggfunc=len).fillna(value=0)
property_types = pd.DataFrame(property_types)

property_types['total'] = property_types.sum(axis=1)
#property_types = property_types.loc[:, 'Apartment':'Villa'].div(property_types.total, axis = 0)
# --->>> JSON
property_types.to_json('property_types.json', orient='index')